In [14]:
# imports
from pathlib import Path
import pandas as pd
import plotly.express as px
# these imports change the graph design
import plotly.io as pio
pio.templates.default = "simple_white"
# imports for accessing our data warehouse
from google.cloud import bigquery
# import and set environment variable
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\SamanthaBrimberry\OneDrive - SIA Partners\Desktop\data_science\civil-hope-323521-3fb9894cd173.json"
# import to ignore warning output
import warnings
warnings.filterwarnings('ignore')

# Load in the Data Via CSV
```python
from pathlib import Path
path = Path(r"C:\Users\SamanthaBrimberry\OneDrive - SIA Partners\Desktop\data_science\GoogleBigQuery_attritionClassification\reporting\data\data.csv")
df = pd.read_csv(path)
```

# Create our SQL Pipeline (very simple)

In [30]:
# client object
project_id = 'civil-hope-323521'
client = bigquery.Client(project=project_id)

# query our entire data living in google cloud
df = client.query('''
select * 
from `civil-hope-323521.attrition_dataset_1.IBM_attrition_2021` # project_id.database.table
''').to_dataframe()

# show the first 5 rows
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,48,False,Travel_Rarely,530,Sales,29,1,Medical,1,473,...,2,80,1,15,3,1,0,0,0,0
1,45,False,Travel_Frequently,364,Research & Development,25,3,Medical,1,1306,...,3,80,0,22,4,3,0,0,0,0
2,41,False,Travel_Rarely,896,Sales,6,3,Life Sciences,1,298,...,3,80,0,16,3,3,1,0,0,0
3,57,False,Travel_Rarely,210,Sales,29,3,Marketing,1,568,...,3,80,1,32,3,2,1,0,0,0
4,42,False,Travel_Rarely,265,Sales,5,2,Marketing,1,1029,...,2,80,0,21,3,4,1,0,0,0


# Exploratory Analysis of Attrition 

In [18]:
# Show distribution for most important columns when predicting Attrition True
# The feature importance CSV is included in the reporting directory
column_feature_top10 = df[['OverTime','JobRole','BusinessTravel','DistanceFromHome','MaritalStatus','NumCompaniesWorked','YearsSinceLastPromotion','Attrition']]

# Loop through every column in the dataframe and plot
'''
for i in column_feature_top10[0:]:
    fig = px.histogram(df,x=i)
    fig.show()
'''

'\nfor i in column_feature_top10[0:]:\n    fig = px.histogram(df,x=i)\n    fig.show()\n'

In [70]:
# init the figures 
fig1 = px.histogram(column_feature_top10,x='OverTime',color='Attrition',barmode='group',title="Count of Over Time where Attrition is True")
fig2 = px.histogram(column_feature_top10, x='BusinessTravel', color='Attrition', barmode='group', title='Level of Business Travel where Attrition is True')
fig3 = px.histogram(column_feature_top10, x='JobRole', color='Attrition', barmode='group', title='Headcount by Department Split by Attrition Status')
fig4 = px.histogram(df, x='MaritalStatus',color='Attrition',barmode='group',title='Count of Marital Status by Attrition')
fig5 = px.histogram(df,x="EnvironmentSatisfaction",color="Attrition",barmode='group',title="Environment Satisfaction by Attrition Status")
fig6 = px.bar(df.query("Gender=='Female'"),x='DailyRate',y='Attrition',color='Department',orientation='h',title="Female Employee's Daily Rate by Attrition Status")
fig7 = px.bar(df.query("Gender=='Male'"),x='DailyRate',y='Attrition',color='Department',orientation='h',title="Male Employee's Daily Rate by Attrition Status")

# store the figure objects into a dict  
figure_dict = {'fig1':fig1,'fig2':fig2,'fig3':fig3,'fig4':fig4,'fig5':fig5,'fig6':fig6,'fig7':fig7}

""" count = 1
text = "fig"
file_names = []
while count <= 7:
    file_names.append(text + str(count))
    count += 1 """
px.
# loop through the figure dict and a) show the output; b) write the figure to a png
for i, x in figure_dict.items():
    x.show()
    x.write_image(f'report_attrition/{i}.png')

In [37]:
# Show distribution for most important columns when predicting Attrition False
# The feature importance CSV is included in the reporting directory
column_feature_bottom10 = df[['JobLevel','WorkLifeBalance','JobInvolvement','EnvironmentSatisfaction','StockOptionLevel','JobRole']]
# Loop through every column in the dataframe and plot
"""for i in column_feature_bottom10[0:]:
    fig = px.histogram(df,x=i)
    fig.show()"""

'for i in column_feature_bottom10[0:]:\n    fig = px.histogram(df,x=i)\n    fig.show()'

# Company Deomgraphics

In [71]:
# Transform data frame
avg_profit = df[['Department','JobRole','MonthlyIncome','MonthlyRate','Attrition','Gender','EducationField','Education','Age','DailyRate']]
avg_profit['MonthlyProfit'] = avg_profit.apply(lambda x: x['MonthlyRate'] - x['MonthlyIncome'],axis=1)
avg_profit['ProfitPCT'] = avg_profit.apply(lambda x: (x['MonthlyProfit'] / sum(avg_profit['MonthlyProfit']))*100, axis=1)
avg_profit['MonthlyRatePCT'] = avg_profit.apply(lambda x: (x['MonthlyRate']/sum(avg_profit['MonthlyRate'])*100),axis=1)
mean_profit_slice = avg_profit.groupby(avg_profit['Department']).mean().round(2)

# init the figures
fig1 = px.histogram(avg_profit,x=avg_profit['ProfitPCT'],y='Department',color='Gender',title='Distribution of Profit as Percent by Department and Gender')
fig2 = px.histogram(avg_profit,x=avg_profit['ProfitPCT'],color='Department',title='Distribution of Profit as Percent by Department')
fig3 = px.histogram(avg_profit,x=avg_profit['ProfitPCT'],y='Attrition',color='Department',title='Distribution of Profit as Percent by Attrition and Department')
fig4 = px.histogram(avg_profit,x="MonthlyRatePCT", y='Department',color='EducationField',title='Monthly Rate as Percentage by Department and Education Field')
fig5 = px.bar(mean_profit_slice,x=mean_profit_slice.index,y='MonthlyProfit',title="Average Monthly Profit by Department")
fig6 = px.scatter(avg_profit, x="MonthlyProfit", y='Age',color='Department', size="DailyRate", title="Distribution of Monthly Profit by Employee Age")

# store the figure objects into a list  
figure_dict = {'fig1':fig1,'fig2':fig2,'fig3':fig3,'fig4':fig4,'fig5':fig5,'fig6':fig6}

# loop through the figure list and print
for i, x in figure_dict.items():
    x.show()
    x.write_image(f'report_company/{i}.png')
